### Вычисление косинусного подобия на абстрактных данных

Реализуем вычисление косинусного подобия в виде функции cos_sim(counts_dict1, counts_dict2)

In [ ]:
from collections import Counter
import math
import re

num = numerator; denom = denominator; dct.values() = частоты в словаре

In [ ]:
def cos_sim(dct1, dct2):
    num = 0
    used = []
    for key1 in dct1:
        for key2 in dct2:
            if key1 in dct2 and key1 not in used:
                num += dct1[key1] * dct2[key1]
                used.append(key1)
                
    denom1, denom2 = 0, 0
    for i in dct1.values():
        denom1 += i * i
    for i in dct2.values():
        denom2 += i * i
    denom = math.sqrt(denom1)*math.sqrt(denom2)
    
    return num/denom

In [ ]:
cos_sim(Counter((1, 0, 0)), Counter((1, 6, 1)))

In [ ]:
cos_sim(Counter((0, 1, 2)), Counter((1, 6, 1)))

In [ ]:
cos_sim(Counter((1, 0, 0)), Counter((0, 1, 2)))

In [ ]:
cos_sim(Counter(('a', 'hh', 'hh')), Counter(('hh', 'a', 'z')))

### Вычисление косинусного подобия предложений, представленных в виде мешка слов

Функция bag_of_words представляет предложение в виде мешка слов

In [ ]:
def bag_of_words(text):
    return re.findall('\w+', text.lower())

In [ ]:
sen1 = 'The cat is on the mat.'
sen2 = 'The cat is on the chair.'
sen3 = 'I bought a new chair.'

In [ ]:
bow1 = Counter(bag_of_words(sen1))
bow2 = Counter(bag_of_words(sen2))
bow3 = Counter(bag_of_words(sen3))
print(bow1)
print(bow2)
print(bow3)

In [ ]:
cos_sim(bow1, bow2)

In [ ]:
cos_sim(bow1, bow3)

In [ ]:
cos_sim(bow2, bow3)

### Реализуем PPMI для матрицы слово-контекст 
#### PPMI = Positive Pointwise Mutual (= взаимный, обоюдный) Information scores
#### PMI = Pointwise Mutual Information scores

In [41]:
import math

In [42]:
dct = \
{'апельсин': {'вкусный': 1,
              'данные': 0,
              'компьютер': 0,
              'результат': 0,
              'сладкий': 1},
 'информация': {'вкусный': 0,
                'данные': 6,
                'компьютер': 1,
                'результат': 4,
                'сладкий': 0},
 'цифровой': {'вкусный': 0,
              'данные': 1,
              'компьютер': 2,
              'результат': 1,
              'сладкий': 0},
 'яблоко': {'вкусный': 1,
            'данные': 0,
            'компьютер': 1,
            'результат': 0,
            'сладкий': 1}}


total_counts = сумма всех частот

In [43]:
def total(dct):
    total_counts = 0
    for d in dct.values():
        for key in d:
            total_counts += d[key]
    return total_counts 

p_wc = вероятность встречи слово-контекст; это - числитель дроби PMI

В знаменателе дроби - произведение вероятности слова на вероятность контекста. 

Вероятность слова (p_w) рассчитывается как его общая частота (во всех контекстах), делённая на сумму всех частот в матрице. 

Вероятность контекста (p_c) рассчитывается как общая частота контекста, делённая на сумму всех частот в матрице. 

In [44]:
def word_probability(word, context, dct, total_counts):
    p_w = 0.0
    for key in dct[word]:
        p_w += dct[word][key]
    p_w /= total_counts
    return p_w

In [45]:
def context_probability(word, context, dct, total_counts):
    p_c = 0.0
    for d in dct.values():
        p_c += d[context]
        #print(d)
    p_c /= total_counts
    return p_c

PPMI отличается только тем, что негативные значения PMI заменяются на 0

In [46]:
def find_pmi(word, context, dct, total_counts):
    p_wc = dct[word][context] / total_counts
    
    p_w = word_probability(word, context, dct, total_counts)
    p_c = context_probability(word, context, dct, total_counts)
    
    pmi = math.log2(p_wc / (p_w * p_c))
    if pmi < 0:
        ppmi = 0
    else:
        ppmi = pmi
    return (pmi, ppmi)

In [47]:
#word, context = "информация", "данные"
word, context = "информация", "результат"
#word, context = "информация", "результат"
#word, context = "информация", "компьютер"

In [48]:
total_counts = total(dct)
#print('word =', word, '; context =', context)
print('query =', word, context)
PMI, PPMI = find_pmi(word, context, dct, total_counts)
print('PMI =', PMI)
print('PPMI =', PPMI)

query = информация результат
PMI = 0.5405683813627027
PPMI = 0.5405683813627027


### Реализуем сглаживание Add-2. 
#### Для этого нужно к числу в каждой ячейке прибавить 2

In [49]:
dct = \
{'апельсин': {'вкусный': 1,
              'данные': 0,
              'компьютер': 0,
              'результат': 0,
              'сладкий': 1},
 'информация': {'вкусный': 0,
                'данные': 6,
                'компьютер': 1,
                'результат': 4,
                'сладкий': 0},
 'цифровой': {'вкусный': 0,
              'данные': 1,
              'компьютер': 2,
              'результат': 1,
              'сладкий': 0},
 'яблоко': {'вкусный': 1,
            'данные': 0,
            'компьютер': 1,
            'результат': 0,
            'сладкий': 1}}

In [50]:
for d in dct.values():
    for key in d:
        d[key] += 2
dct

{'апельсин': {'вкусный': 3,
  'данные': 2,
  'компьютер': 2,
  'результат': 2,
  'сладкий': 3},
 'информация': {'вкусный': 2,
  'данные': 8,
  'компьютер': 3,
  'результат': 6,
  'сладкий': 2},
 'цифровой': {'вкусный': 2,
  'данные': 3,
  'компьютер': 4,
  'результат': 3,
  'сладкий': 2},
 'яблоко': {'вкусный': 3,
  'данные': 2,
  'компьютер': 3,
  'результат': 2,
  'сладкий': 3}}

In [51]:
#word, context = "информация", "данные"
word, context = "информация", "результат"
#word, context = "информация", "результат"
#word, context = "информация", "компьютер"

In [53]:
total_counts = total(dct)
#print(total_counts)
#print('word =', word, '; context =', context)
print('query =', word, context)
PMI, PPMI = find_pmi(word, context, dct, total_counts)
print('PMI =', PMI)
print('PPMI =', PPMI)

query = информация результат
PMI = 0.3990959554098223
PPMI = 0.3990959554098223
